In [1]:
import os  
import sys
import pathlib

repo_path = pathlib.Path('.').absolute().parent
src_path = repo_path / 'src'

sys.path.insert(0, str(src_path)) 

In [2]:
import pymongo

In [23]:
import fairscape_mds.mds.config as mds_config

server_config = mds_config.get_fairscape_config('../deploy/local.env')

server_config.setup_mongo()

server_config.setup_minio()

In [ ]:
mongoClient = server_config.CreateMongoClient()

mongoDB = mongoClient[server_config.mongo.db]

mongoUserCollection = mongoDB[server_config.mongo.user_collection]

In [28]:
result = mongoUserCollection.find_one({"@id": "not a user"})

In [30]:
result is None

True

## API Requests

In [31]:
import requests
fairscape_host = "http://localhost:8080/"

## Users

In [104]:
# creating a user
user_metadata = {
    "@id": "ark:59852/jdoe",
    "@type": "Person",
    "name": "John Doe",
    "email": "jdoe@example.org",
    "password": "examplepassword"
}

create_user_response = requests.post(
    fairscape_host + 'user',
    json=user_metadata
    )

In [105]:
print(create_user_response.status_code)
print(create_user_response.content)

201
b'{"created":{"@id":"ark:59852/jdoe","@type":"Person","name":"John Doe"}}'


In [106]:
# get a user
get_user_response = requests.get(
    fairscape_host + f'user/{user_metadata.get("@id")}'
)

In [107]:
print(get_user_response.status_code)
print(get_user_response.content)

200
b'{"@id":"ark:59852/jdoe","@type":"Person","@context":{"@vocab":"https://schema.org/","evi":"https://w3id.org/EVI#"},"url":null,"organizations":[],"projects":[],"datasets":[],"rocrates":[],"software":[],"computations":[],"evidencegraphs":[],"name":"John Doe","email":"jdoe@example.org","password":"examplepassword"}'


In [108]:
# get a user that doestn't exist
get_user_response = requests.get(
    fairscape_host + 'user/ark:99999/fakeuser'
)

In [109]:
print(get_user_response.status_code)
print(get_user_response.content)

404
b'{"error":"No record found"}'


In [110]:
# list users
list_user_response = requests.get(
    fairscape_host + 'user'
)

In [111]:
print(list_user_response.status_code)
print(list_user_response.content)

200
b'{"users":[{"@id":"ark:59852/softwaretester","@type":"Person","name":"John Doe"},{"@id":"ark:59852/jdoe","@type":"Person","name":"John Doe"}]}'


In [112]:
# delete user
delete_user_response = requests.delete(
    fairscape_host +  f'user/{user_metadata.get("@id")}'
)

In [113]:
print(delete_user_response.status_code)
print(delete_user_response.content)

200
b'{"deleted":{"@id":"ark:59852/jdoe","@type":"Person","name":"John Doe","email":"jdoe@example.org","datasets":[],"software":[],"computatations":[],"rocrates":[],"evidencegraphs":[]}}'


## Dataset

In [154]:
# creating a user
test_user_metadata = {
    "@id": "ark:59852/softwaretester",
    "@type": "Person",
    "name": "John Doe",
    "email": "jdoe@example.org",
    "password": "examplepassword"
}

create_user_response = requests.post(
    fairscape_host + 'user',
    json=test_user_metadata
    )

In [155]:
# create a dataset

test_dataset_metadata = {
    "@id": "ark:59852/test-dataset",
    "@type": "evi:Dataset",
    "name": "test dataset",
    "description": "an example metadata set for a test dataset",
    "keywords": [ "test", "fair"],
    "owner": test_user_metadata['@id'],
    "author": "John Doe"
}

create_dataset_response = requests.post(
    fairscape_host + 'dataset',
    json = test_dataset_metadata
)

print(create_dataset_response.status_code)
print(create_dataset_response.content)

400
b'{"error":"dataset already exists"}'


In [156]:
# list datasets
list_dataset_response = requests.get(
    fairscape_host + 'dataset'
)

print(list_dataset_response.status_code)
print(list_dataset_response.content)

200
b'{"datasets":[{"@id":"ark:59852/test-dataset","@type":"evi:Dataset","name":"test dataset"}]}'


In [157]:
# get dataset
get_dataset_response = requests.get(
    fairscape_host + f'dataset/{test_dataset_metadata["@id"]}'
)

print(get_dataset_response.status_code)
print(get_dataset_response.content)

200
b'{"@id":"ark:59852/test-dataset","@context":{"@vocab":"https://schema.org/","evi":"https://w3id.org/EVI#"},"url":null,"license":" https://creativecommons.org/licenses/by/4.0/","keywords":["test","fair"],"distribution":[],"includedInDataCatalog":null,"sourceOrganization":null,"author":"John Doe","dateCreated":"2024-04-08T16:30:55.437000","dateModified":"2024-04-08T16:30:55.437000","usedBy":[],"@type":"evi:Dataset","name":"test dataset","description":"an example metadata set for a test dataset","owner":"ark:59852/softwaretester"}'


In [ ]:
# update dataset

In [118]:
# delete dataset
delete_dataset_response = requests.delete(
    fairscape_host + f'dataset/{test_dataset_metadata["@id"]}'
)

print(delete_dataset_response.status_code)
print(delete_dataset_response.content)

200
b'{"deleted":{"@id":"ark:59852/test-dataset","@type":"evi:Dataset","name":"test dataset"}}'


## Software

In [134]:
# create software
test_software_metadata = {
    "@id": "ark:59852/test-software",
    "@type": "evi:Software",
    "owner": test_user_metadata.get('@id'),
    "name": "Test Software",
    "author": "Max Levinson",
    "keywords": ['evi'],
    "description": "an example software model"
}

In [137]:
# create a dataset

create_software_response = requests.post(
    fairscape_host + 'software',
    json = test_software_metadata
)

print(create_software_response.status_code)
print(create_software_response.content)

201
b'{"created":{"@id":"ark:59852/test-software","metadataType":"evi:Software","name":"Test Software","@context":{"@vocab":"https://schema.org/","evi":"https://w3id.org/EVI#"},"url":null,"description":"an example software model","license":" https://creativecommons.org/licenses/by/4.0/","keywords":["evi"],"owner":"ark:59852/softwaretester","distribution":[],"usedBy":[],"sourceOrganization":null,"includedInDataCatalog":null,"@type":"evi:Software","author":"Max Levinson"}}'


In [138]:
# delete software
delete_software_response = requests.delete(
    fairscape_host + 'software/' + test_software_metadata.get("@id")
)

print(delete_software_response.status_code)
print(delete_software_response.content)



500
b'{"error":"software not deleted"}'


## Computation

In [ ]:
# create computation

## DataDownload

In [ ]:
# create data download

## ROCrate

In [ ]:
# create rocrate

## Resolver

In [153]:
get_user_resolver = requests.get(
    fairscape_host + test_user_metadata.get("@id")
)

print(get_user_resolver.status_code)
print(get_user_resolver.content)

NameError: name 'test_user_metadata' is not defined

In [143]:
get_software_resolver = requests.get(
    fairscape_host + test_software_metadata.get("@id")
)

print(get_software_resolver.status_code)
print(get_software_resolver.content)

404
b'{"detail":"Not Found"}'


In [152]:
get_dataset_resolver = requests.get(
    fairscape_host + test_dataset_metadata.get("@id")
)

print(get_dataset_resolver.status_code)
print(get_dataset_resolver.content)

500
b'{"error":"error returning ark metadata","message":"Object of type datetime is not JSON serializable","identifier":"ark:59852/test-dataset","metadata":"{\'@id\': \'ark:59852/test-dataset\', \'@type\': \'evi:Dataset\', \'name\': \'test dataset\', \'@context\': {\'@vocab\': \'https://schema.org/\', \'evi\': \'https://w3id.org/EVI#\'}, \'url\': None, \'description\': \'an example metadata set for a test dataset\', \'license\': \' https://creativecommons.org/licenses/by/4.0/\', \'keywords\': [\'test\', \'fair\'], \'owner\': \'ark:59852/softwaretester\', \'distribution\': [], \'includedInDataCatalog\': None, \'sourceOrganization\': None, \'author\': \'John Doe\', \'dateCreated\': datetime.datetime(2024, 4, 8, 16, 30, 55, 437000), \'dateModified\': datetime.datetime(2024, 4, 8, 16, 30, 55, 437000), \'usedBy\': []}"}'


In [ ]:
get_user_resolver = requests.get(
    fairscape_host + '/' + testing_user_metadata.get("@id")
)

print(get_user_resolver.status_code)
print(get_user_resolver.content)